In [ ]:
%matplotlib inline

import time
import numpy as np
#import progressbar_renamed as progressbar
import copy
import networkx as nx
from operator import itemgetter
from dimod import BinaryQuadraticModel
from dimod import Vartype

from dimod.reference.samplers import SimulatedAnnealingSampler
import neal

import import_ipynb
import QAOA_for_QLS as qaoa
import Side_Calc_GB2 as sc

In [ ]:
#IBMQ.enable_account(token=...)

In [2]:
def top_gains_populate_subset(G, subset_size, gains): #function for population subset, alternative approaches would be possible here
    if G.number_of_nodes() <= subset_size:
        return list(G.nodes())
    return [
        x[0] for x in sorted(gains.items(), key=itemgetter(1))[-subset_size:]
    ]

In [3]:
def iteration_step(G, B, curr_solution, subset, method='SimAnneal', backend='simulator', graph2subgraph=None): #local optimization for subset and fixed boardering vertices
    
    nn=len(subset)
    subset2global = dict((x, subset[x]) for x in range(nn))
    global2subset = dict((subset[x], x) for x in range(nn))
    
    print(graph2subgraph)
    
    C = [0.0] * nn #create linear term representing fixed vertices
    for i in subset:
        for j in set(G.nodes()) - set(subset):
            C[global2subset[i]] += 2 * B[graph2subgraph[i], graph2subgraph[j]] * curr_solution[graph2subgraph[j]]
    
    indices = []
    for i in subset:
        indices.append(graph2subgraph[i])
    indices=np.array(indices)  # rows and columns of B to keep for subset
    
    
    if method=='SimAnneal':
            st=time.time()
            for i in range(nn):
                C[i]=(-1)*C[i]
            quadr={}
            s = ['']*nn
            for i in range(nn):
                s[i]="s_"+str(i)
            linear=dict((s[i],C[i]) for i in range(nn))
            
            for i in range(nn):
                for j in range(nn):
                    quadr[(s[i],s[j])]=(-1)*B[graph2subgraph[subset2global[i]], graph2subgraph[subset2global[j]]]

            bqm = BinaryQuadraticModel(linear, quadr, 0, Vartype.BINARY)
            
            sampler = neal.SimulatedAnnealingSampler()
            response = sampler.sample(bqm)
            
            ss=0
            i=0
            en=0
            conf_classic=[]

            for datum in response.data(['sample', 'energy']):   
                if en>datum.energy:
                        en=datum.energy
                        ss=i
                con=[]
                for h in range(nn):
                    con.append(datum.sample[s[h]])

                conf_classic.append(con)
                    
                result=np.array(conf_classic[ss])
                if(i >= 10):
                    break
                i += 1
            et=time.time()
            ex_time=(et-st)*1000 #time in milliseconds
            optimized_subset=result
            
            
            
    elif method=='QAOA':     
        result = qaoa.qaoa_basic(len(subset), B[np.ix_(indices, indices)],C, backend)
        optimized_subset=result[0].x
        ex_time=result[1]
    
        
    if 0 in optimized_subset:
        optimized_subset = [
            -1 if x == 0 else 1 if x == 1 else 'Error' for x in optimized_subset
        ]
        
    for i in range(0, len(optimized_subset)):
        curr_solution[graph2subgraph[subset2global[i]]] = optimized_subset[i]
    return curr_solution, ex_time



def QLS(G,size_of_iteration=12,stopping_criteria=3, method='SimAnneal', backend='simulator'):
    B = nx.modularity_matrix(G, nodelist=sorted(G.nodes()), weight='weight')
    
    lis=list(G.nodes())
    n=len(lis)
    
    subgraph2graph=dict((x,lis[x]) for x in range (n)) #necessary for recursive version
    graph2subgraph=dict((lis[x],x) for x in range (n))
    
    #print(graph2subgraph)
    
    # random initial guess
    curr_solution = [
        1 - 2 * x
        for x in list(np.random.randint(2, size=n))
    ]
    
    curr_solution=np.array(curr_solution)
    if 0 in curr_solution:
        curr_solution = [
            -1 if x == 0 else 1 if x == 1 else 'Error' for x in curr_solution
        ]
    curr_modularity = sc.compute_modularity(G, B, curr_solution)
    #print("curr_mod", curr_modularity, "Curr_sol", curr_solution)
    
    all_time_best_solution = curr_solution
    all_time_best_modularity = curr_modularity

    visited = set()
    it = 0
    it_stuck = 0
    all_modularities = []
    ex_time=[]
    
    while set(G.nodes()) - visited:
        it += 1
        if it_stuck > stopping_criteria: 
            break
        gains_list = []
        for v in progressbar.progressbar(G.nodes()):
            gains_list.append(sc.compute_gain(G, B, curr_solution, v,True,graph2subgraph))
        gains = {v: gain for gain, v in gains_list} 
        
        subset = top_gains_populate_subset(G, size_of_iteration, gains) #identify most promising subset based on gains
        
        iter_step = iteration_step(G, B, copy.deepcopy(curr_solution), list(subset), method, backend, graph2subgraph)  #optimize for subset and fixed outer-subset vertices
        cand_solution=iter_step[0]
        #print("cand_sol",cand_solution)
        ex_time.append(iter_step[1])
        cand_modularity = sc.compute_modularity(G, B, cand_solution)
        
        #print('it', it, 'cand_modularity', cand_modularity, 'curr_best',
        #      all_time_best_modularity)
        all_modularities.append({
            'it': it,
            'cand_modularity': cand_modularity,
            'curr_best': all_time_best_modularity
        })
        if cand_modularity > curr_modularity: #update if (modified/unnormalized) modularity has improved
            curr_solution = cand_solution
            curr_modularity = cand_modularity
            it_stuck = 0
        else:
            it_stuck += 1
        if curr_modularity > all_time_best_modularity:
            all_time_best_solution = curr_solution
            all_time_best_modularity = curr_modularity
        
        #this version does not include an additional break-condition in case a very good solution has been found but rather keeps improving until it_stuck>3
    time=(np.array(ex_time)).sum()
    return (time, all_time_best_modularity, all_time_best_solution, it,
            all_modularities)
        